quick start: http://docs.h5py.org/en/latest/quick.html

Dimension Scales: http://docs.h5py.org/en/latest/high/dims.html

In [1]:
import h5py

In [2]:
f = h5py.File('test.hdf5', mode='w')  # can also be used as context manager

In [3]:
f.name

'/'

In [4]:
f['a'] = 12

In [5]:
list(f.keys())

['a']

In [6]:
data = f['a']
data

<HDF5 dataset "a": shape (), type "<i8">

In [7]:
data[...]

array(12)

In [8]:
data.shape

()

In [9]:
list(data.dims)

[]

In [10]:
data.dtype

dtype('int64')

In [11]:
data.maxshape

()

In [12]:
data.name

'/a'

In [13]:
data.attrs['extra info'] = 'super data'

In [14]:
dict(data.attrs.items())

{'extra info': 'super data'}

In [15]:
dict(f.items())

{'a': <HDF5 dataset "a": shape (), type "<i8">}

Dimension Scales

http://docs.h5py.org/en/latest/high/dims.html

In [16]:
import numpy as np

In [17]:
numpy_array = np.array([[1., 2, 3], [4, 5, 6]])

In [18]:
f['b'] = numpy_array
f['b']

<HDF5 dataset "b": shape (2, 3), type "<f8">

In [19]:
f.create_dataset('c', dtype='float32', data=numpy_array)

<HDF5 dataset "c": shape (2, 3), type "<f4">

In [20]:
dict(f.items())

{'a': <HDF5 dataset "a": shape (), type "<i8">,
 'b': <HDF5 dataset "b": shape (2, 3), type "<f8">,
 'c': <HDF5 dataset "c": shape (2, 3), type "<f4">}

In [21]:
f['b'].dims[1].label = 'nice second dimension!'

In [22]:
f['myscale'] = [1, 2, 3, 4]

In [23]:
# this seems to be only necessary for setting a label:
f['c'].dims.create_scale(f['myscale'], 'my scale label')
#h5py.h5ds.set_scale(f['myscale'].id, b'my scale label')

In [24]:
f['c'].dims[0].attach_scale(f['myscale'])

In [25]:
[dim.label for dim in f['b'].dims]

['', 'nice second dimension!']

In [26]:
list(f['c'].dims.keys())

[<"" dimension 0 of HDF5 dataset at 140378890863152>,
 <"" dimension 1 of HDF5 dataset at 140378890863152>]

In [27]:
list(f['c'].dims[0].keys())

['my scale label']

In [28]:
f['c'].dims[0][0]

<HDF5 dataset "myscale": shape (4,), type "<i8">

In [29]:
f['c'].dims[0]['my scale label']

<HDF5 dataset "myscale": shape (4,), type "<i8">

In [30]:
f.close()

In [31]:
!h5dump test.hdf5

HDF5 "test.hdf5" {
GROUP "/" {
   DATASET "a" {
      DATATYPE  H5T_STD_I64LE
      DATASPACE  SCALAR
      DATA {
      (0): 12
      }
      ATTRIBUTE "extra info" {
         DATATYPE  H5T_STRING {
            STRSIZE H5T_VARIABLE;
            STRPAD H5T_STR_NULLTERM;
            CSET H5T_CSET_UTF8;
            CTYPE H5T_C_S1;
         }
         DATASPACE  SCALAR
         DATA {
         (0): "super data"
         }
      }
   }
   DATASET "b" {
      DATATYPE  H5T_IEEE_F64LE
      DATASPACE  SIMPLE { ( 2, 3 ) / ( 2, 3 ) }
      DATA {
      (0,0): 1, 2, 3,
      (1,0): 4, 5, 6
      }
      ATTRIBUTE "DIMENSION_LABELS" {
         DATATYPE  H5T_STRING {
            STRSIZE H5T_VARIABLE;
            STRPAD H5T_STR_NULLTERM;
            CSET H5T_CSET_ASCII;
            CTYPE H5T_C_S1;
         }
         DATASPACE  SIMPLE { ( 2 ) / ( 2 ) }
         DATA {
         (0): NULL, "nice second dimension!"
         }
      }
   }
   DATASET "c" {
      DATATYPE  H5T_IEEE_F32LE
      DATASPAC

In [32]:
with h5py.File('two_scales.hdf5', mode='w') as f:
    f['a'] = [1, 2, 3]
    f['scale1'] = 1
    f['scale2'] = 2
    f['a'].dims.create_scale(f['scale1'], 'scale one')
    h5py.h5ds.set_scale(f['scale2'].id, b'scale two')
    f['a'].dims[0].attach_scale(f['scale1'])
    f['a'].dims[0].attach_scale(f['scale2'])

In [33]:
!h5dump two_scales.hdf5

HDF5 "two_scales.hdf5" {
GROUP "/" {
   DATASET "a" {
      DATATYPE  H5T_STD_I64LE
      DATASPACE  SIMPLE { ( 3 ) / ( 3 ) }
      DATA {
      (0): 1, 2, 3
      }
      ATTRIBUTE "DIMENSION_LIST" {
         DATATYPE  H5T_VLEN { H5T_REFERENCE { H5T_STD_REF_OBJECT }}
         DATASPACE  SIMPLE { ( 1 ) / ( 1 ) }
         DATA {
         (0): (DATASET 1400 /scale1 , DATASET 1672 /scale2 )
         }
      }
   }
   DATASET "scale1" {
      DATATYPE  H5T_STD_I64LE
      DATASPACE  SCALAR
      DATA {
      (0): 1
      }
      ATTRIBUTE "CLASS" {
         DATATYPE  H5T_STRING {
            STRSIZE 16;
            STRPAD H5T_STR_NULLTERM;
            CSET H5T_CSET_ASCII;
            CTYPE H5T_C_S1;
         }
         DATASPACE  SCALAR
         DATA {
         (0): "DIMENSION_SCALE"
         }
      }
      ATTRIBUTE "NAME" {
         DATATYPE  H5T_STRING {
            STRSIZE 10;
            STRPAD H5T_STR_NULLTERM;
            CSET H5T_CSET_ASCII;
            CTYPE H5T_C_S1;
         }
